# Evaluate optimization generation with distributed workers

In [1]:
using FUSE
using JLD2
using Plots;

## Define run directories

In [2]:
top_dir = "exploratory/"
run_dir = "opt_runs3/"
gen_eval = 300

result_paths = [
    "opt_betaN_cost__Solovev_Kr_flattop24_HTS0.1_A3.5_Zeff2.0/",
    ]

1-element Vector{String}:
 "opt_betaN_cost__Solovev_Kr_flattop24_HTS0.1_A3.5_Zeff2.0/"

## Setup distributed computing environment

In [3]:
old_dir = pwd()
mkpath(top_dir)
try
    cd(top_dir) 
    FUSE.parallel_environment("omega", 128*2) 
finally
    cd(old_dir)
end
display(pwd())
using Distributed
#@everywhere import WarmupFUSE
@everywhere import IJulia
@everywhere import FUSE

connecting to worker 1 out of 256
connecting to worker 2 out of 256
connecting to worker 3 out of 256
connecting to worker 4 out of 256
connecting to worker 5 out of 256
connecting to worker 6 out of 256
connecting to worker 7 out of 256
connecting to worker 8 out of 256
connecting to worker 9 out of 256
connecting to worker 10 out of 256
connecting to worker 11 out of 256
connecting to worker 12 out of 256
connecting to worker 13 out of 256
connecting to worker 14 out of 256
connecting to worker 15 out of 256
connecting to worker 16 out of 256
connecting to worker 17 out of 256
connecting to worker 18 out of 256
connecting to worker 19 out of 256
connecting to worker 20 out of 256
connecting to worker 21 out of 256
connecting to worker 22 out of 256
connecting to worker 23 out of 256
connecting to worker 24 out of 256
connecting to worker 25 out of 256
connecting to worker 26 out of 256
connecting to worker 27 out of 256
connecting to worker 28 out of 256
connecting to worker 29 out o

"/fusion/ga/projects/ird/ptp/weisbergd/julia/dev/FUSE/playground/weisberg_FPP_opt"

## Define main loop function

In [3]:
result_path = result_paths[1]

# load opt_ini file
@load joinpath(top_dir, result_path, "opt_ini.jld2") opt_ini

params = opt_ini["free_parameters"]
ini, act = FUSE.case_parameters(:FPP, version=:v1_demount, init_from=:scalars);

# sort run dirs
cache_path = joinpath(top_dir*result_path, run_dir)

all_dirs = filter(isdir, sort(readdir(cache_path; join=true)))
println(length(all_dirs))

dirs = sort(filter(x -> !isfile(joinpath(x, "error.txt")), all_dirs))
println(length(dirs))

# load optimization results file(s)
con = []

for i in range(1,6)

    fname = "result"*string(i)*".jld2"

    try
        @load joinpath(top_dir, result_path, fname) result
        display("Loaded "*fname)
        for c in result.convergence
            con = [con; c]
        end
    catch
        display("Could not find "*fname)
    end

end

# get population input parameters for ith generation
X = [p.x for p in con[300].population];
X = reduce(hcat, X)';

# load ini and act from optimization dir
dd, ini, act = FUSE.load(dirs[end]);

ini1 = deepcopy(ini);

25344
25221


"Loaded result1.jld2"

"Loaded result2.jld2"

"Loaded result3.jld2"

"Loaded result4.jld2"

...

"Could not find result5.jld2"

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/cHcDY/src/JLD2.jl:287


"Could not find result6.jld2"

┌ Error: reading act.ActorDivertors.impurities : TypeError(:setfield!, "", Union{Missing, Vector{Symbol}}, Float64[])
└ @ SimulationParameters ~/.julia/dev/SimulationParameters/src/utils.jl:92


FUSE.ParametersInits{Float64}
├─ general
│  ├─ casename ➡ "FPP_v1_demount_scalars" Mnemonic name of the case being run
│  └─ init_from ➡ :scalars Initialize run from [:ods, :scalars]
├─ gasc
│  ├─ filename ➡ "/fusion/ga/projects/ird/ptp/weisbergd/julia/dev/FUSE/cases/../sample/FPPv1.0_aspectRatio3.5_PBpR35_demount.json"
│  │  Output GASC .json file from which data will be loaded
│  └─ case ➡ 0 Number of the GASC run to load
├─ ods
│  └─ filename ➡ missing ODS.json file from which equilibrium is loaded
├─ build
│  ├─ layers ➡ OrderedCollections.OrderedDict{Symbol, Float64}(
│  │           :gap_OH => 0.9660441468573052
│  │           :OH => 0.519738179067469
│  │           :gap_TF_OH => 0.07343221127763265
│  │           :hfs_TF => 0.519738179067469
│  │           :hfs_gap_low_temp_shield_TF => 0.07343221127763265
│  │           :hfs_low_temp_shield => 0.31814263905127016
│  │           :hfs_gap_vacuum_vessel_low_temp_shield => 0.07343221127763265
│  │           :hfs_vacuum_vessel => 0.1

In [9]:
using SimulationParameters

x = [1.0,2.0,3.0,4.0,5.0]

display(ini.core_profiles)

ini1.core_profiles.greenwald_fraction = ini1.core_profiles.greenwald_fraction ↔ [-Inf, Inf]

SimulationParameters.parameters_from_opt!(ini1, x)

ini1.core_profiles

core_profiles{Float64}
├─ greenwald_fraction ➡ 0.8138940843944762 Line average electron density expressed as fraction of Greenwald density
├─ greenwald_fraction_ped ➡ 0.5372962948090667 Pedestal electron density expressed as fraction of Greenwald density
├─ ne_ped ➡ missing Pedestal electron density
├─ w_ped ➡ 0.05 Pedestal width expressed in fraction of ψₙ
├─ T_ratio ➡ 0.825 Ti/Te ratio
├─ T_shaping ➡ 2.5 Temperature shaping factor
├─ n_shaping ➡ 2.5 Density shaping factor
├─ zeff ➡ 2.0 Effective ion charge
├─ rot_core ➡ 0.0 [s^-1] Derivative of the flux surface averaged electrostatic potential with respect to the poloidal flux,
│  multiplied by -1. This quantity is the toroidal angular rotation frequency due to the ExB drift, introduced in formula
│  (43) of Hinton and Wong, Physics of Fluids 3082 (1985), also referred to as sonic flow in regimes in which the toroidal
│  velocity is dominant over the poloidal velocity
├─ ngrid ➡ 101 Resolution of the core_profiles grid
├─ bulk ➡ :DT 

core_profiles{Float64}
├─ greenwald_fraction ➡ 1.0 Line average electron density expressed as fraction of Greenwald density
├─ greenwald_fraction_ped ➡ 0.5372962948090667 Pedestal electron density expressed as fraction of Greenwald density
├─ ne_ped ➡ missing Pedestal electron density
├─ w_ped ➡ 0.05 Pedestal width expressed in fraction of ψₙ
├─ T_ratio ➡ 0.825 Ti/Te ratio
├─ T_shaping ➡ 2.5 Temperature shaping factor
├─ n_shaping ➡ 2.5 Density shaping factor
├─ zeff ➡ 2.0 Effective ion charge
├─ rot_core ➡ 0.0 [s^-1] Derivative of the flux surface averaged electrostatic potential with respect to the poloidal flux,
│  multiplied by -1. This quantity is the toroidal angular rotation frequency due to the ExB drift, introduced in formula
│  (43) of Hinton and Wong, Physics of Fluids 3082 (1985), also referred to as sonic flow in regimes in which the toroidal
│  velocity is dominant over the poloidal velocity
├─ ngrid ➡ 101 Resolution of the core_profiles grid
├─ bulk ➡ :DT Bulk ion specie

In [12]:
using ProgressMeter

function evaluate_runs(top_dir, result_path, i)

    # sort run dirs
    cache_path = joinpath(top_dir*result_path, run_dir)

    all_dirs = filter(isdir, sort(readdir(cache_path; join=true)))
    println(length(all_dirs))

    dirs = sort(filter(x -> !isfile(joinpath(x, "error.txt")), all_dirs))
    println(length(dirs))

    # load optimization results file(s)
    con = []

    for i in range(1,6)

        fname = "result"*string(i)*".jld2"
    
        try
            @load joinpath(result_path, fname) result
            display("Loaded "*fname)
            for c in result.convergence
                con = [con; c]
            end
        catch
            display("Could not find "*fname)
        end
    
    end

    # get population input parameters for ith generation
    X = [p.x for p in con[i].population];
    X = reduce(hcat, X)';

    # load ini and act from optimization dir
    dd, ini, act = FUSE.load(dirs[end]);

    # re-assign optimization variables in ini
    ini.core_profiles.greenwald_fraction = ini.core_profiles.greenwald_fraction ↔ [0.85, 1.3]
    ini.equilibrium.B0 = ini.equilibrium.B0 ↔ [1.0, 10.0]
    ini.equilibrium.ip = ini.equilibrium.ip ↔ [1.0e6, 20.0e6]
    ini.equilibrium.R0 = ini.equilibrium.R0 ↔ [1.0, 15.0]
    ini.equilibrium.pressure_core = ini.equilibrium.pressure_core ↔ [0.1e6, 3.0e6];
    
    # define objective functions and constraints
    objective_functions = opt_ini["objective_functions"]
    constraint_functions = opt_ini["constraint_functions"]

    ## Run evaluation function
    save_folder = result_path * "gen" * string(i)
    println(top_dir*save_folder)
    try 
        mkdir(top_dir*save_folder)
    catch 
        println("folder already exists")
    end

    p = ProgressMeter.Progress(1; desc="Iteration", showspeed=true)
    @time FUSE.optimization_engine(ini, act, my_workflow, X, objective_functions, constraint_functions, save_folder, true, p);

end


evaluate_runs (generic function with 1 method)

## Define opt workflow function

In [13]:
# re-define opt workflow

@everywhere function my_workflow(ini::FUSE.ParametersInits, act::FUSE.ParametersActors)

    ### modify ini layers
    
    R0 = ini.equilibrium.R0
    ϵ = ini.equilibrium.ϵ
    a = R0 * ϵ

    RiPlasma = R0 - a
    
    # HFS layers

    ini.build.layers[:plasma] = 2*a*(1+ini.build.plasma_gap)
    ini.build.layers[:hfs_first_wall] = 0.02
    ini.build.layers[:hfs_blanket] = 0.55*((1/ϵ-1.8)/2.2)^0.5
    ini.build.layers[:hfs_high_temp_shield] = RiPlasma*0.05
    ini.build.layers[:hfs_gap_high_temp_shield_vacuum_vessel] = RiPlasma * 0.02
    ini.build.layers[:hfs_vacuum_vessel] = RiPlasma * 0.05
    ini.build.layers[:hfs_gap_vacuum_vessel_low_temp_shield] = RiPlasma * 0.02
    ini.build.layers[:hfs_low_temp_shield] = 0.45-0.15*((1/ϵ-1.8)/2.2)^0.5
    ini.build.layers[:hfs_gap_low_temp_shield_TF] = RiPlasma * 0.02

    rbCenterPost = (R0 - 0.5*ini.build.layers[:plasma] - ini.build.layers[:hfs_first_wall] - ini.build.layers[:hfs_blanket]
        - ini.build.layers[:hfs_high_temp_shield] - ini.build.layers[:hfs_gap_high_temp_shield_vacuum_vessel]
        - ini.build.layers[:hfs_vacuum_vessel] - ini.build.layers[:hfs_gap_vacuum_vessel_low_temp_shield]
        - ini.build.layers[:hfs_low_temp_shield] - ini.build.layers[:hfs_gap_low_temp_shield_TF]);

    ini.build.layers[:hfs_TF] = rbCenterPost * 0.25
    ini.build.layers[:OH] = rbCenterPost * 0.25
    ini.build.layers[:gap_TF_OH] = RiPlasma * 0.02
    ini.build.layers[:gap_OH] = rbCenterPost - ini.build.layers[:hfs_TF] - ini.build.layers[:gap_TF_OH] - ini.build.layers[:OH]

    # LFS layers

    ini.build.layers[:lfs_first_wall] = 0.02
    ini.build.layers[:lfs_blanket] = 1.0 - 0.25*((1/ϵ-2.1)/1.9)^0.5
    ini.build.layers[:lfs_high_temp_shield] = RiPlasma * 0.05
    ini.build.layers[:lfs_gap_high_temp_shield_vacuum_vessel] = RiPlasma * 0.2
    ini.build.layers[:lfs_vacuum_vessel] = RiPlasma * 0.05
    ini.build.layers[:lfs_gap_vacuum_vessel_low_temp_shield] = RiPlasma * 0.02
    ini.build.layers[:lfs_low_temp_shield] = 0.45-0.15*((1/ϵ-1.8)/2.2)^0.5
    ini.build.layers[:lfs_gap_low_temp_shield_TF] = RiPlasma * 0.02
    ini.build.layers[:lfs_TF] = ini.build.layers[:hfs_TF]
    
    ### run init
    
    dd = FUSE.init(ini, act)
    
    ### run main compound actor
    
    FUSE.ActorWholeFacility(dd, act)
    
    return dd
end

## Loop through result_paths and evaluate

In [14]:
for result_path in result_paths
    
    println(top_dir*result_path)
    evaluate_runs(top_dir,result_path,gen_eval)
    println("")

    
end

nominal/opt_betaN_cost__Solovev_Kr_flattop24_HTS0.1_qpol2.75_A3.5_Zeff2.0/
30464
30172
nominal/opt_betaN_cost__Solovev_Kr_flattop24_HTS0.1_qpol2.75_A3.5_Zeff2.0/gen300


┌ Error: reading act.ActorDivertors.impurities : TypeError(:setfield!, "", Union{Missing, Vector{Symbol}}, Float64[])
└ @ SimulationParameters ~/.julia/dev/SimulationParameters/src/utils.jl:92


128.000380 seconds (1.05 M allocations: 49.491 MiB)



## Release computing resources

In [12]:
# when done, release workers
if true
    for i in workers()
        rmprocs(i)
    end
end